In [1]:
# %matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import signal
import matplotlib.animation as animation
import os
import pickle

'''set parameter'''

samplerate = 5              # サンプリングレート[fps]
fp = 0.2                      # 通過域端周波数[Hz] 2
fs = 2                      # 阻止域端周波数[Hz] 3
gpass = 1                   # 通過域最大損失量[dB] 1
gstop = 2                   # 阻止域最小減衰量[dB] 40

'''---------------------------'''

# butterworth low-pass filter
def lowpass(x, samplerate, fp, fs, gpass, gstop):
    fn = samplerate / 2                           #ナイキスト周波数
    wp = fp / fn                                  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn                                  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    y = signal.filtfilt(b, a, x)                  #信号に対してフィルタをかける
    return y        

# get data
data = []
# number = 1 # 1 to 22
for number in range(1,23):
#     print(number)
    name = 'peregrine_' +  "%02d" % number + '.xlsx'
    filename = '/Users/tsutsuikazushi/Documents/Nagoya/animals/data/peregrine/' + name
    xlsx_data = pd.read_excel(filename, encoding="utf_8", sep=",")
    xlsx_data = np.array(xlsx_data)

    
    # print(40000 / 360)
    # 1 (deg) : 111111 (m) = data (deg) : x (m) 
    # x (m) = 111111 * data: 

    # pp = Peregrine Position
    pp1 = xlsx_data[:,0] * 111111
    pp2 = xlsx_data[:,1] * 111111
    pp3 = xlsx_data[:,2]

    # tp = Target Position
    tp1 = xlsx_data[:,4] * 111111
    tp2 = xlsx_data[:,5] * 111111
    tp3 = xlsx_data[:,6]

    # low-pass filter
    pp1 = lowpass(pp1, samplerate, fp, fs, gpass, gstop)
    pp2 = lowpass(pp2, samplerate, fp, fs, gpass, gstop)
    pp3 = lowpass(pp3, samplerate, fp, fs, gpass, gstop)
    tp1 = lowpass(tp1, samplerate, fp, fs, gpass, gstop)
    tp2 = lowpass(tp2, samplerate, fp, fs, gpass, gstop)
    tp3 = lowpass(tp3, samplerate, fp, fs, gpass, gstop)
    
    # shift target position
    pp1_end = pp1[-1]
    pp2_end = pp2[-1]
    pp3_end = pp3[-1]
    
    tp1_end = tp1[-1]
    tp2_end = tp2[-1]
    tp3_end = tp3[-1]
    
    dis1_end = pp1_end - tp1_end
    dis2_end = pp2_end - tp2_end
    dis3_end = pp3_end - tp3_end
    
    tp1 += dis1_end 
    tp2 += dis2_end
    tp3 += dis3_end
    
    # shift to origin
    x_min = np.min([np.min(pp1), np.min(tp1)])
    x_max = np.max([np.max(pp1), np.max(tp1)])
    y_min = np.min([np.min(pp2), np.min(tp2)])
    y_max = np.max([np.max(pp2), np.max(tp2)])
    z_min = np.min([np.min(pp3), np.min(tp3)])
    z_max = np.max([np.max(pp3), np.max(tp3)]) 

    pp1 -= x_min
    pp2 -= y_min
    pp3 -= z_min
    tp1 -= x_min 
    tp2 -= y_min
    tp3 -= z_min

    # calculate velocity
    dpp1 = np.diff(pp1)
    dpp2 = np.diff(pp2)
    dpp3 = np.diff(pp3)
    dtp1 = np.diff(tp1)
    dtp2 = np.diff(tp2)
    dtp3 = np.diff(tp3)

    # ajust position data length to velocity data
    pp1 = pp1[:-1]
    pp2 = pp2[:-1]
    pp3 = pp3[:-1]
    tp1 = tp1[:-1]
    tp2 = tp2[:-1]
    tp3 = tp3[:-1]
    
    print('pp1', pp1)
    print('pp2', pp2)
    print('pp3', pp3)

    # get speed from the original data file
    ps = xlsx_data[:-1,3]
    ts = xlsx_data[:-1,7]

    p_data = []
    p_data.append(dpp1); p_data.append(dpp2); p_data.append(dpp3)
    p_data.append(pp1); p_data.append(pp2); p_data.append(pp3)
    p_data.append(ps)
    
    t_data = []
    t_data.append(dtp1); t_data.append(dtp2); t_data.append(dtp3)
    t_data.append(tp1); t_data.append(tp2); t_data.append(tp3)
    t_data.append(ts)
    
    data_seq = []
    data_seq.append(p_data)
    data_seq.append(t_data)
    
    data.append(data_seq)
    
print(np.array(p_data).shape, np.array(t_data).shape, np.array(data_seq).shape, np.array(data).shape)
    
np.save('peregrine_data', data)
   

(7, 100) (7, 100) (2, 7, 100) (22, 2, 7, 100)


In [2]:
import numpy as np

data = np.load('peregrine_data.npy', allow_pickle=True)

print(np.array(data).shape)



(22, 2, 7, 100)
